In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
from transformers import BertTokenizer, BertModel
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score, roc_curve, auc)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Dense, Dropout, BatchNormalization, Embedding, Flatten, Input, Concatenate)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [10]:
# Funkce pro načtení a spojení dat
def load_and_merge_data(train_path, test_path, submit_path):
    train_df = pd.read_csv(train_path, sep=';')
    test_df = pd.read_csv(test_path, sep=';')
    submit_df = pd.read_csv(submit_path, sep=';')
    test_df = pd.concat([test_df, submit_df], axis=0, ignore_index=True).drop(columns=["idS"], errors='ignore')
    df_all = pd.concat([train_df, test_df], axis=0, ignore_index=True)
    return df_all

# Funkce pro předzpracování dat pomocí BERT a imputace
def preprocess_data(df):
    target_column = df.columns[-1]
    X = df.drop(columns=[target_column])
    Y = df[target_column]

    # Imputace chybějících numerických hodnot
    imputer = SimpleImputer(strategy='mean')
    X_num_imputed = pd.DataFrame(imputer.fit_transform(X.select_dtypes(include=np.number)), columns=X.select_dtypes(include=np.number).columns)

    # Použití BERT pro textové zpracování
    if 'title' in df.columns:
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Používáme předtrénovaný BERT tokenizer
        model = BertModel.from_pretrained('bert-base-uncased')  # Používáme předtrénovaný BERT model

        # Ujistíme se, že 'title' obsahuje pouze textové hodnoty (řetězce)
        titles = df['title'].astype(str).fillna('')

        def get_bert_embeddings(texts):
            # Kontrola, že texts je seznam řetězců
            if isinstance(texts, str):
                texts = [texts]
            inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)  # Tensors pro BERT model
            with torch.no_grad():  # Deaktivujeme výpočet gradientů pro rychlejší inference
                outputs = model(**inputs)
            return outputs.last_hidden_state.mean(dim=1).numpy()  # Průměr posledních skrytých stavů jako embeddingy

        X_title_bert = get_bert_embeddings(titles.tolist())  # Vytváříme embeddingy pro texty
    else:
        X_title_bert = np.zeros((X.shape[0], 768))  # BERT embeddingy mají rozměr 768

    # Standardizace numerických dat
    scaler = StandardScaler()
    X_num_scaled = scaler.fit_transform(X_num_imputed)

    # Sloučení numerických a textových dat
    X_final = np.hstack([X_num_scaled, X_title_bert])

    # Label encoding a one-hot encoding pro cílovou proměnnou
    label_encoder = LabelEncoder()
    Y_encoded = label_encoder.fit_transform(Y)
    one_hot_encoder = OneHotEncoder(sparse_output=False)
    Y_onehot = one_hot_encoder.fit_transform(Y_encoded.reshape(-1, 1))

    return X_final, Y_onehot

# Funkce pro vytvoření modelu
def create_model(input_shape, num_classes):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(input_shape,), kernel_regularizer=tf.keras.regularizers.l2(0.0005)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005)),
        BatchNormalization(),
        Dropout(0.4),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Načtení a předzpracování dat
df_all = load_and_merge_data('train (2).csv', 'test (1).csv', 'evaluation.csv')
X, Y = preprocess_data(df_all)

# Rozdělení dat
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Vytvoření a trénování modelu
model = create_model(X_train.shape[1], y_train.shape[1])
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)
history = model.fit(X_train, y_train, epochs=40, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr])

RuntimeError: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 9849977856 bytes.

In [ ]:
# Vizualizace průběhu trénování
from sklearn.calibration import label_binarize


plt.figure(figsize=(12, 4))

# Ztráta
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], "ro-", label="Training loss")
plt.plot(history.history["val_loss"], "r-", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()

# Přesnost
plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], "bo-", label="Training accuracy")
plt.plot(history.history["val_accuracy"], "b-", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()

plt.show()

# Predikce na testovacích datech
y_pred_probs = model.predict(X_test)  # Pravděpodobnosti tříd
y_pred = y_pred_probs.argmax(axis=1)  # Převod na nejpravděpodobnější třídu
y_true = y_test.argmax(axis=1)  # Skutečné hodnoty

# Klasifikační report
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# ROC křivka a AUC skóre
y_true_bin = label_binarize(y_true, classes=[i for i in range(y_test.shape[1])])  # One-hot encoding pro skutečné třídy
plt.figure(figsize=(6, 6))

for i in range(y_test.shape[1]):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
    auc_score = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {auc_score:.3f})')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("ROC Curve")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Predikce na testovacích datech
y_pred_probs = model.predict(X_test)  # Pravděpodobnosti tříd
y_pred = y_pred_probs.argmax(axis=1)  # Převod na nejpravděpodobnější třídu
y_true = y_test.argmax(axis=1)  # Skutečné hodnoty

# Výpočet metrik
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Výpis metrik
print(f'🔹 Přesnost: {accuracy:.4f}')
print(f'🔹 Precision: {precision:.4f}')
print(f'🔹 Recall: {recall:.4f}')
print(f'🔹 F1-score: {f1:.4f}')
